# Predicting the next Tennis Tournament Winner's country

For this project we are using he dataset containing the information about the tennis players from varoups countries to develop a predictive model for forecasting the next tournament winner's country. The data set includes multiple varibles like winner's height, age, rank, country, loser's country age, height and match statistics. 

The data captures diverse data from 2017 to 2019, so in this project by grouping winners based in their countries and historical tournament data, we aim to identify patterns and trends that can predict about the future tournament outcomes. 

First we will read the data using various libraries and functions.

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(readr)
library(dplyr)
library(rsample)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
# Read the dataset from the web into R
tennis_players <- read_csv("data/tennis_players.csv")
# Preprocess data
tournament_players <- tennis_players |>
mutate(winner_country=as.factor(winner_ioc))|>
drop.na(winner_ht)

New names:
• `` -> `...1`
Rows: 6866 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in drop.na(mutate(tennis_players, winner_country = as.factor(winner_ioc)), : could not find function "drop.na"


In [ ]:
# Group winners by country and calculate statistics
winner_stats <- tournament_players |>
  group_by(winner_country) |>
  summarize(avg_age = mean(winner_age, na.rm = TRUE), 
            avg_height = mean(winner_ht, na.rm = TRUE),
            total_wins = n())|>
filter(!is.na(avg_height))
winner_stats

In [ ]:
# Split the dataset into training and testing sets
library(rsample)
set.seed(123) # for reproducibility
data_split <- initial_split(winner_stats, prop = 0.8)
data_train <- training(data_split)
data_test <- testing(data_split)

In [ ]:
tennis_recipe <- recipe(winner_country ~ avg_age + avg_height+total_wins, data = data_train)|>
step_center(all_predictors())|>
step_scale(all_predictors())

tennis_recipe

In [ ]:
# Specify the k-NN model
knn_model <- nearest_neighbor(weight_func = "rectangular", neighbors = tune())|> 
  set_engine("kknn") |>
  set_mode("classification")
knn_model

In [ ]:
# Create the workflow
workflow <- workflow() |>
  add_recipe(tennis_recipe) |>
  add_model(knn_model)
workflow

In [ ]:
# Train the model
model <- fit(workflow, data_train)
model

In [ ]:
# Make predictions on the test set
predictions <- predict(model, data_test)|>
bind_cols(data_test)
predictions
# Evaluate model performance
players_test_metrics <- predictions |>
  metrics(truth = winner_country, estimate = .pred_class)
players_test_precision <- predictions |>
  precision(truth = winner_country, estimate = .pred_class)
players_test_recall <- predictions |>
  recall(truth = winner_country, estimate = .pred_class)
players_test_conf_matrix <- predictions |>
  conf_mat(truth = winner_country, estimate = .pred_class)

# Display evaluation metrics and confusion matrix
players_test_metrics
players_test_precision
players_test_recall
players_test_conf_matrix

In [ ]:
set.seed(2020)


players_test_metrics <- predictions |>
  metrics(truth = winner_country, estimate = .pred_class)
#filter(.metric=="accuracy")
players_test_metrics
# Precision
players_test_precision <- predictions |>
  precision(truth = winner_country, estimate = .pred_class, event_level="first")
players_test_precision
# Recall
players_test_recall <- predictions |>
  recall(truth = winner_country, estimate = .pred_class, event_level="first")
players_test_recall

# Confusion Matrix
players_test_conf_matrix <- predictions |>
  conf_mat(truth = winner_country, estimate=.pred_class)
players_test_conf_matrix